Plot Antarctic Tide Range
=========================

This ({nb-download}`notebook <Plot-Antarctic-Tide-Range.ipynb>`) demonstrates plotting the total combined tidal amplitude for all constituents around Antarctica

```{important}
*Need to download tide model prior to running this notebook.*  
```

OTIS format tidal solutions provided by Oregon State University and ESR  
- [http://volkov.oce.orst.edu/tides/region.html](http://volkov.oce.orst.edu/tides/region.html) 
- [https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/](https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/)
- [ftp://ftp.esr.org/pub/datasets/tmd/](ftp://ftp.esr.org/pub/datasets/tmd/)  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  
- [https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models](https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models)

Finite Element Solution (FES) provided by AVISO  
- [https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html)
        
## Python Dependencies
 - [cartopy: Python package designed for geospatial data processing](https://scitools.org.uk/cartopy/docs/latest/)  
 - [h5netcdf: Pythonic interface to netCDF4 via h5py](https://h5netcdf.org/)
 - [matplotlib: Python 2D plotting library](http://matplotlib.org/)  
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [xarray: N-D labeled arrays and datasets in Python](https://docs.xarray.dev/en/stable/) 

## Program Dependencies

- `io.model.py`: retrieves tide model parameters for named tide models  
- `io.OTIS.py`: extract tidal harmonic constants from OTIS tide models  
- `io.ATLAS.py`: extract tidal harmonic constants from ATLAS netcdf models  
- `io.GOT.py`: extract tidal harmonic constants from GOT tide models  
- `io.FES.py`: extract tidal harmonic constants from FES tide models  

```{note}
This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.  
```

## Load modules

In [ ]:
import numpy as np
import xarray as xr
import matplotlib
matplotlib.rcParams['axes.linewidth'] = 2.0
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# import tide programs
import pyTMD.io
import pyTMD.tools
import timescale

# autoreload
%load_ext autoreload
%autoreload 2

## Set parameters for program

- Model directory  
- Tide model  

In [ ]:
# available model list
model_list = sorted(pyTMD.io.model.ocean_elevation())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'GOT4.10_nc'
TMDwidgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
])

## Read tide model

In [ ]:
# get model parameters
model = pyTMD.io.model(TMDwidgets.directory.value,
   ).from_database(TMDwidgets.model.value)
ds = model.open_dataset(type='z')

## Setup coordinates for calculating tides

In [ ]:
# create an image around Antarctica
xlimits = [-560.*5e3,560.*5e3]
ylimits = [-560.*5e3,560.*5e3]
spacing = [20e3,-20e3]
# x and y coordinates
x = np.arange(xlimits[0],xlimits[1]+spacing[0],spacing[0])
y = np.arange(ylimits[1],ylimits[0]+spacing[1],spacing[1])
xgrid,ygrid = np.meshgrid(x,y)
# x and y dimensions
nx = int((xlimits[1]-xlimits[0])/spacing[0])+1
ny = int((ylimits[0]-ylimits[1])/spacing[1])+1
# convert data to coordinate reference system of model
X, Y = ds.tmd.transform(xgrid, ygrid, crs=3031)
# convert to data arrays
X = xr.DataArray(X, dims=('y','x'))
Y = xr.DataArray(Y, dims=('y','x'))

## Calculate tide map

In [ ]:
# interpolate to grid and convert to centimeters
local = ds.tmd.interp(X, Y)
local = local.tmd.to_units('cm')
# tide min and max
tide_min = xr.DataArray(np.inf*np.ones((ny,nx)), coords=local.coords, dims=local.dims)
tide_max = xr.DataArray(-np.inf*np.ones((ny,nx)), coords=local.coords, dims=local.dims)
# calculate tide maps over a span greater than 18.6 year nodal cycle
for year in range(2000, 2021):
    # calculate tides over year
    ts = timescale.from_range(f'{year}-01-01', f'{year}-12-31', step=1, units='h')
    tide = local.tmd.predict(ts.tide, corrections=model.corrections)
    tide += local.tmd.infer(ts.tide, corrections=model.corrections)
    # update tide min and max
    tide_min = xr.where(tide.min(axis=0) < tide_min, tide.min(axis=0), tide_min)
    tide_max = xr.where(tide.max(axis=0) > tide_max, tide.max(axis=0), tide_max)
# calculate tidal range
tide_range = tide_max - tide_min

## Create plot of tidal range

In [ ]:
# plot Antarctic tide range
projection = ccrs.Stereographic(central_longitude=0.0,
    central_latitude=-90,true_scale_latitude=-71.0)
fig, ax = plt.subplots(num=1, figsize=(9,8),
    subplot_kw=dict(projection=projection))
# plot tide height
vmin,vmax = (0, np.max(tide_range))
extent = (xlimits[0],xlimits[1],ylimits[0],ylimits[1])
im = ax.imshow(tide_range, interpolation='nearest',
    vmin=vmin, vmax=vmax, transform=projection,
    extent=extent, origin='upper')
# add high resolution cartopy coastlines
ax.coastlines('10m')

# Add colorbar and adjust size
# pad = distance from main plot axis
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax, pad=0.025, extend='max',
    extendfrac=0.0375, shrink=0.85, aspect=22.5, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_ylabel(f'{model.name} Tide Range', labelpad=10, fontsize=13)
cbar.ax.set_title('cm', fontsize=13, va='bottom')
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=20,
    labelsize=13, direction='in')

# set x and y limits
ax.set_xlim(xlimits)
ax.set_ylim(ylimits)

# stronger linewidth on frame
ax.spines['geo'].set_linewidth(2.0)
ax.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
# show the plot
plt.show()